In [1]:
import os
from pathlib import Path
import xml.etree.ElementTree as ET

from tqdm import tqdm
import numpy as np
import math
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt

import cv2

from utils import (
    yolo2pixel,
    nms_yv1,
    get_bboxes,
    mAP,
    plot_preds)

INFO:numexpr.utils:Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


# Folders

In [2]:
ds_dir = 'pets/test/'

### PETS

In [3]:
val_imgs = ds_dir + 'images/'
val_labels = ds_dir + 'labels/'

print(f'DFire val dir: {len(os.listdir(val_imgs))}')

DFire val dir: 368


# Config

In [4]:
CLASSES = ["cat", "dog"]

IMG_DIM = {'W':224, 'H':224} # (W, H)
IMG_W = IMG_DIM['W']
IMG_H = IMG_DIM['H']

SX = 7
SY = 7
B = 2 # Number of bounding boxes to predict.
C = len(CLASSES) # Number of classes in the dataset.

BATCH_SIZE = 64
WORKERS = 2

# PETS Dataset

In [5]:
class PETS(Dataset):
    '''
    Creates a Pytorch Dataset to train the Yolov1 Network.
    Encodes labels to match the format [xcell, ycell, w, h, confidence, class_0 (smoke), class_1 (fire)]
        - Final encoding format is: [xcell, ycell, w, h, conf=1, smoke?, fire?]

    Discard images when there are more than 1 object in the same cell
    
    Arguments:
        - img_h:            image height
        - img_w:            image width
        - img_dir:          path to images folder
        - label_dir:        path to labels folder
        - SX:               number of cells in X axis (horizontal -> width)
        - SY:               number of cells in Y axis (vertical -> height)
        - C:                number of classes, 2 in this case
        - transform:        transformation applied to input images -> Albumentations
        - target_transform: transformation applied to labels -> nothing by default

    Return:
        - img:              1 image of the dataset
        - target:           corresponding label encoded
    '''

    def __init__(self, img_h, img_w, img_dir, label_dir, 
                 SX, SY, C, 
                 transform=None, target_transform=None):
        self.img_h = img_h
        self.img_w = img_w
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.SX = SX
        self.SY = SY
        self.C = C
        self.transform = transform
        self.target_transform = target_transform

        self.labels_list = sorted(
            [
                os.path.join(self.label_dir, file_name)
                for file_name in os.listdir(self.label_dir)
                if file_name.endswith(".txt")
            ]
        )
        
        self.images, self.bboxes, self.labels = self.__build_ds__(self.labels_list)
        
        self.num_samples = self.images.shape[0]

    def __len__(self):
        return self.num_samples
    
    def __bbox_check__(self, bbox):
        eps = 1e-6
        
        xc, yc, w, h = bbox[0], bbox[1], bbox[2], bbox[3]
        xmin = xc - w/2
        ymin = yc - h/2
        xmax = xc + w/2
        ymax = yc + h/2
        
        xmin = max(xmin, 0 + eps)
        ymin = max(ymin, 0 + eps)
        xmax = min(xmax, 1)
        ymax = min(ymax, 1)
        
        bbox = np.array([ 
                (xmin+xmax)/2,
                (ymin+ymax)/2,
                xmax-xmin,
                ymax-ymin
                 ]).astype(np.float32)
        
        return bbox        


    def __build_ds__(self, labels_list):
        bboxes = []
        labels = []
        images = []
        wrong_imgs = 0
        overlapping_rem = 0
        more_than_3 = 0
                
        for label in labels_list:
            fname = Path(label).stem
            image_path = self.img_dir + fname + '.jpg'   
            #print(fname, image_path)
                                   
            if cv2.imread(image_path) is None:
                print(f'{image_path} cannot be read by cv2 -> removed')
                wrong_imgs += 1
            
            else:
                
                label_mtx = np.zeros((self.SY, self.SX))
                overlapping_object = 0

                one_bboxes = []
                one_labels = []
            
                with open(label) as f:
                    lines = f.readlines()

                    # Restrict to 3 boxes per sample
                    if len(lines) > 3:
                        more_than_3 += 1
                        #continue
                        
                    for line in lines:
                        class_id, x, y, w, h = line.strip().split()
                        class_id = int(class_id)
                        box = np.array([x, y, w, h]).astype(np.float32)
                        x, y, w, h = box[0], box[1], box[2], box[3]
                        box_ok = self.__bbox_check__([x, y, w, h])
                        x, y, w, h = box_ok[0], box_ok[1], box_ok[2], box_ok[3]
                        i, j = math.floor(y * self.SY), math.floor(x * self.SX)
                        if label_mtx[i, j] == 1:
                            overlapping_object = 1
                            overlapping_rem += 1
                            #print(f'Removed {label} due to overlapping object in cell {i, j}')
                            break
                        else:
                            label_mtx[i, j] = 1
                            one_bboxes.append([x, y, w, h])
                            # smoke
                            if class_id == 0:
                                one_labels.append(0)
                            # fire
                            elif class_id == 1:
                                one_labels.append(1)
                            else:
                                print(f'File {label} errored in cell {i, j}')

                    if overlapping_object == 0:
                        # Padding to SX*SY labels and bounding boxes, so you can store tensors
                        # Label -1 indicates no box
                        for idx in range(self.SX*self.SY - len(one_labels)):
                            one_bboxes.append([0, 0, 0, 0])
                            one_labels.append(-1)
                        # print(f'\nBboxes and Labels of image {image_path}')
                        # print("Bboxes")
                        # for box in one_bboxes:
                        #     print(box)
                        # print("Labels")
                        # for label in one_labels:
                        #     print(label)
                        bboxes.append(one_bboxes)
                        labels.append(one_labels)
                        images.append(image_path)
        
        print(f'Removed wrong images: {wrong_imgs}')
        print(f'Removed due to overlapping: {overlapping_rem}')
        print(f'More than 3: {more_than_3}')
 
        labels_np = np.array(labels)
        labels_tensor = torch.tensor(labels_np, dtype=torch.float32)
        bboxes_np = np.array(bboxes)
        bboxes_tensor = torch.tensor(bboxes_np, dtype=torch.float32)
        images_array = np.array(images)
        # print(f'Images array {images_array}')
        # print(f'Bboxes tensor {bboxes_tensor}')
        # print(f'Labels tensor {labels_tensor}')
        
        return images_array, bboxes_tensor, labels_tensor
        #return images, bboxes, labels

    def __getitem__(self, index):

        # Image processing
        img_file = self.images[index]
        img = cv2.imread(img_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   
        #img = cv2.resize(img, (self.img_w, self.img_h), interpolation = cv2.INTER_NEAREST)

        # Labels processing
        bboxes = self.bboxes[index]
        bboxes = bboxes[~torch.all(bboxes == torch.tensor([0,0,0,0]), dim=1)]
        bboxes = bboxes.numpy().tolist()
        #print(bboxes)
        labels = self.labels[index]
        labels = labels[labels != -1.]
        labels = labels.numpy().tolist()
        #print(f'Labels inside dataset {labels}')
        
        # Data Augmentation
        if self.transform is not None:
            try:
                aug = self.transform(image=img, bboxes=bboxes, class_labels=labels)
                img = aug['image']
                bboxes = aug['bboxes']
                labels = aug['class_labels']
            except:
                #print(f'Error trying to augment image {img_file}')
                img = cv2.resize(img, (self.img_w, self.img_h), interpolation = cv2.INTER_NEAREST)
                img = (img / 255.)
                img = torch.tensor(img, dtype=torch.float32)
                img = img.permute(2, 0, 1)
        
        label_mtx = np.zeros((self.SY, self.SX, 5+self.C))
        
        for box, label in zip(bboxes, labels):
            class_id = int(label)
            i, j = int(box[1]*self.SY), int(box[0]*self.SX)
            xcell, ycell = box[0]*self.SX - j, box[1]*self.SY - i
            label_mtx[i, j, :5] = [xcell, ycell, box[2], box[3], 1]
            label_mtx[i, j, 5+class_id] = 1

        label_mtx = torch.tensor(label_mtx, dtype=torch.float32)
        
        #return img, label_mtx, img_file
        return img, label_mtx

# Preprocess, Datasets and DataLoaders

In [6]:
val_transform = A.Compose([
    A.Resize(IMG_H, IMG_W, p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1),
    ToTensorV2(p=1),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

val_dataset = PETS(img_h = IMG_H,
                   img_w = IMG_W,
                   img_dir = val_imgs,
                   label_dir = val_labels,
                   SX = SX,
                   SY = SY,
                   C = C,
                   transform=val_transform)


val_loader = DataLoader(dataset=val_dataset,
                        batch_size=BATCH_SIZE,
                        num_workers=WORKERS,
                        pin_memory=True,
                        shuffle=False,
                        drop_last=True)

Removed wrong images: 0
Removed due to overlapping: 0
More than 3: 0


# Model

In [7]:
class OPTIM_BED(nn.Module):
    def __init__(self, num_classes, S, B, in_channels=3):
        super(OPTIM_BED, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.S = S
        self.B = B
        
        self.model = self.__create_BED__()

        
    def __create_BED__(self):
        BED_model = nn.Sequential(
            # Conv2d [in_channels, out_channels, kernel_size, stride, padding, bias]

            # CNNBlock 224x224
            nn.Conv2d(self.in_channels, 64, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            nn.Dropout2d(p=0.5),

            # CNNBlock 112x112
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 24, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(24, affine=False),
            nn.ReLU(),
            nn.Dropout2d(p=0.5),

            # CNNBlock 56x56
            nn.MaxPool2d(kernel_size=2, stride=2),
            # kernel = 1 in github
            nn.Conv2d(24, 16, kernel_size=1, stride=1, padding=0,  bias=False),
            nn.BatchNorm2d(16, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            
            # kernel = 1 in github
            nn.Conv2d(32, 32, kernel_size=1, stride=1, padding=0,  bias=False),
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),

            # CNNBlock 28x28
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0,  bias=False),
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0,  bias=False),
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0,  bias=False),
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),

            # CNNBlock 14x14
            nn.MaxPool2d(kernel_size=2, stride=2), 
            nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0,  bias=False),
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0,  bias=False),
            nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            
            # CNNBlock 7x7
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(128, affine=False),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1,  bias=False),
            nn.BatchNorm2d(128, affine=False),
            nn.ReLU(),
            
            # CNNBlock Out
            nn.Conv2d(128, 128, kernel_size=1, stride=1, padding=0,  bias=False),
            nn.BatchNorm2d(128, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(128, 64, kernel_size=1, stride=1, padding=0,  bias=False),
            nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(64, 16, kernel_size=1, stride=1, padding=0,  bias=False),
            nn.BatchNorm2d(16, affine=False),
            nn.ReLU(),
            
            nn.Conv2d(16, self.B*5 + self.num_classes, kernel_size=1, stride=1, padding=0,  bias=False),
            
        )
        return BED_model
        
          

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_in',
                    nonlinearity='relu'
                )
                if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    # [xc1, yc1, w1, h1, conf1, xc2, yc2, w2, h2, conf2, smoke, fire]
    # [0 ................. 4,    5 ................ 9      10    11 ]
    def forward(self, x):
        x_out = self.model(x)
        x = x_out.permute(0, 2, 3, 1)
        if self.B == 1:
            class_softmax = torch.softmax(x[..., 5:7], dim=-1)
            x = torch.cat((torch.sigmoid(x[..., 0:5]), class_softmax), dim=-1)  
        else:
            class_softmax = torch.softmax(x[..., 10:12], dim=-1)
            x = torch.cat((torch.sigmoid(x[..., 0:10]), class_softmax), dim=-1)
        return x 

In [8]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


In [9]:
model = OPTIM_BED(num_classes=C, S=SX, B=B, in_channels=3).to(DEVICE)
model_path = 'results/'
model_file = model_path + 'bed_best_2BB_mAP.pt'

checkpoint = torch.load(model_file, map_location=torch.device(DEVICE))
model.load_state_dict(checkpoint['model_state_dict'])

model.eval() 

OPTIM_BED(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (2): ReLU()
    (3): Dropout2d(p=0.5, inplace=False)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (6): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (7): ReLU()
    (8): Dropout2d(p=0.5, inplace=False)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(24, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (11): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (12): ReLU()
    (13): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (14): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False

# Calculate mAP

In [10]:
cell2box_mask = torch.zeros((SY, SX, 2))
for i in range(SY):
    for j in range(SX):
        cell2box_mask[i,j,0] = j
        cell2box_mask[i,j,1] = i 

In [11]:
def calculate_mAP(thres, iou_nms, iou_map, loader, model, mask):
    pred_boxes, target_boxes = get_bboxes(loader=loader, 
                                      model=model,
                                      SX=SX,
                                      SY=SY,
                                      B=B,
                                      C=C,
                                      mask=mask,
                                      iou_threshold=iou_nms, 
                                      threshold=thres,
                                      device=DEVICE,
                                      box_format="midpoint")

    mean_avg_prec, avg_prec, cls_prec, cls_rec = mAP(pred_boxes=pred_boxes, 
                                                     true_boxes=target_boxes, 
                                                     iou_threshold=iou_map, 
                                                     box_format="midpoint",
                                                     num_classes=C)
    # print(f'mAP: {mean_avg_prec}')
    # print(f'smoke AP: {avg_prec[0]}')
    # print(f'fire AP: {avg_prec[1]}')
    # print(f'smoke precision: {cls_prec[0]}')
    # print(f'fire precision: {cls_prec[1]}')
    # print(f'smoke recall: {cls_rec[0]}')
    # print(f'fire recall: {cls_rec[1]}')

    f1_smoke = 2*(cls_prec[0]*cls_rec[0])/(cls_prec[0]+cls_rec[0])
    f1_fire = 2*(cls_prec[1]*cls_rec[1])/(cls_prec[1]+cls_rec[1])
    f1_mean = (f1_smoke + f1_fire) / 2

    print("mAP".ljust(9) + "|" + 
          "AP smoke".ljust(9) + "|" + 
          "AP fire".ljust(9) + "|" + 
          "F1 mean".ljust(9) + "|" + 
          "F1 smoke".ljust(9) + "|" + 
          "F1 fire".ljust(9) + "|" +
          "Pr smoke".ljust(9) + "|" +
          "Pr fire".ljust(9) + "|" +
          "Re smoke".ljust(9) + "|" +
          "Re fire".ljust(9))

    print_hyphen = ""
    for i in range(10): # 10 = # of metrics
        print_hyphen += "---------".ljust(9) + " "
    print(print_hyphen)
    
    print(f'{mean_avg_prec:.3f}'.ljust(9) + "|" +
          f'{avg_prec[0]:.3f}'.ljust(9) + "|" +
          f'{avg_prec[1]:.3f}'.ljust(9) + "|" +
          f'{f1_mean:.3f}'.ljust(9) + "|" +
          f'{f1_smoke:.3f}'.ljust(9) + "|" +
          f'{f1_fire:.3f}'.ljust(9) + "|" +
          f'{cls_prec[0]:.3f}'.ljust(9) + "|" +
          f'{cls_prec[1]:.3f}'.ljust(9) + "|" +
          f'{cls_rec[0]:.3f}'.ljust(9) + "|" +
          f'{cls_rec[1]:.3f}'.ljust(9) + "\n")


In [12]:
calculate_mAP(thres=0.2,
              iou_nms=0.3,
              iou_map=0.5,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:00<00:00, 29.58it/s]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.681    |0.689    |0.672    |0.755    |0.770    |0.741    |0.764    |0.686    |0.776    |0.805    



In [13]:
calculate_mAP(thres=0.2,
              iou_nms=0.5,
              iou_map=0.5,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:00<00:00, 26.86it/s]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.685    |0.689    |0.680    |0.736    |0.755    |0.717    |0.735    |0.637    |0.776    |0.821    



In [14]:
calculate_mAP(thres=0.1,
              iou_nms=0.5,
              iou_map=0.5,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:00<00:00, 23.39it/s]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.697    |0.710    |0.683    |0.676    |0.711    |0.640    |0.635    |0.523    |0.808    |0.826    



In [15]:
calculate_mAP(thres=0.2,
              iou_nms=0.3,
              iou_map=0.4,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:00<00:00, 31.96it/s]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.763    |0.741    |0.786    |0.807    |0.794    |0.821    |0.787    |0.760    |0.800    |0.892    



In [16]:
calculate_mAP(thres=0.2,
              iou_nms=0.3,
              iou_map=0.3,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:00<00:00, 27.58it/s]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.789    |0.749    |0.829    |0.825    |0.802    |0.849    |0.795    |0.786    |0.808    |0.923    



In [17]:
calculate_mAP(thres=0.1,
              iou_nms=0.5,
              iou_map=0.4,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:00<00:00, 23.49it/s]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.781    |0.763    |0.798    |0.720    |0.732    |0.708    |0.654    |0.578    |0.832    |0.913    



In [18]:
calculate_mAP(thres=0.1,
              iou_nms=0.5,
              iou_map=0.3,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:00<00:00, 23.41it/s]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.801    |0.772    |0.830    |0.730    |0.739    |0.720    |0.660    |0.588    |0.840    |0.928    



In [22]:
calculate_mAP(thres=0.5,
              iou_nms=0.5,
              iou_map=0.5,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:00<00:00, 28.69it/s]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.614    |0.616    |0.612    |0.749    |0.768    |0.731    |0.869    |0.738    |0.688    |0.723    



In [23]:
calculate_mAP(thres=0.4,
              iou_nms=0.5,
              iou_map=0.5,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:00<00:00, 30.17it/s]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.636    |0.636    |0.635    |0.753    |0.771    |0.735    |0.840    |0.717    |0.712    |0.754    



In [24]:
calculate_mAP(thres=0.3,
              iou_nms=0.5,
              iou_map=0.5,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:00<00:00, 28.03it/s]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.671    |0.689    |0.653    |0.761    |0.789    |0.733    |0.802    |0.691    |0.776    |0.779    

